In [ ]:
import xml.sax
import html
class ManejadorNombres(xml.sax.ContentHandler):
    """
    Clase encargada de manejar el ejercicio 1
    """
    def __init__(self):
        """
        metodo constructor
        """
        super().__init__()
        self.is_name=False
        self.names=[]
        self.current_name=""

    def startElement(self, name, attrs):
        """
        Se llama al comienzo de cada elemento del XML
        """
        if name=="name":
            self.is_name=True


    def characters(self, content):
        """
        Se llama para ver el contenido de cada elemento,
        No tiene porque llamarse una sola vez
        """
        if self.is_name:
            self.current_name+=content

    def endElement(self, name):
        """
        Se llama al final de cada elemento del XML
        """
        if name=="name" :
            self.is_name=False
            self.names.append(html.unescape(self.current_name).strip())
            self.current_name=""

    def get_names(self,sorted_list):
        """
        Devuelve una lista con los nombres encontrados en el html
        """
        if sorted_list :
            self.names.sort()
        return self.names


In [ ]:

def nombres_restaurantes(filename):
    """
    Llama a Sax con el manejadorNombres y devuelve la lista de nombres ordenada alfabeticamente
    """
    manejador=ManejadorNombres()
    parser = xml.sax.make_parser()
    parser.setContentHandler(manejador)
    parser.parse(filename)
    return manejador.get_names(True)


for name in nombres_restaurantes("restaurantes_v1_es.xml"):
    print(name)

In [ ]:
class ManejadorSubcategorias(xml.sax.ContentHandler):
    
    def __init__(self):

        self.num_evento = 1
        self.is_categoria=False
        self.is_subcategoria=False
        self.subcategorias=set()
        self.save_category=False
        self.save_subcategory=False
        self.current_category=""
        self.current_subcategory=""
        
    def startElement(self, etiqueta, atributos):
        if(etiqueta=="categoria"):
            self.is_categoria=True
        if(etiqueta=="subcategoria"):
            self.is_subcategoria=True
        if(self.is_subcategoria):
            for _,value in atributos.items():
                if(value=="SubCategoria"):
                    self.save_subcategory=True
        elif(self.is_categoria):
            for _,value in atributos.items():
                if(value=="Categoria"):
                    self.save_category=True

    def characters(self, contenido):
        if(self.save_subcategory):
            self.current_subcategory+=contenido
        elif self.save_category:
            self.current_category+=contenido
            
    def endElement(self, etiqueta):
        if(self.current_subcategory!=""):
            aux=f"{self.current_category} > {self.current_subcategory}"
            self.subcategorias.add(aux)
        if(etiqueta=="categoria"):
            self.is_categoria=False
            self.current_category=""
        if(etiqueta=="subcategoria"):
            self.is_subcategoria=False
            self.current_subcategory=""
        self.save_category=False
        self.save_subcategory=False

    def getSubcategories(self):
        return self.subcategorias

In [ ]:

def subcategorias(filename):
    manejador=ManejadorSubcategorias()
    parser = xml.sax.make_parser()
    parser.setContentHandler(manejador)
    parser.parse(filename)
    return manejador.getSubcategories()


for sub in subcategorias("restaurantes_v1_es.xml"):
    print(sub)


In [3]:
from xml.dom.minidom import parse
import xml.dom.minidom
def rellenarCampo(service, diccionario, campo, xmlName):
    data=None
    if len(service.getElementsByTagName(xmlName)[0].childNodes)>0:
        node=service.getElementsByTagName(xmlName)[0].childNodes[0]
        data=html.unescape(node.data)
    diccionario[campo]=data
def rellenarDiccionario(service,cname):
    dictionary={}
    rellenarCampo(service,dictionary,"name","name")
    rellenarCampo(service,dictionary,"descripcion","body")
    rellenarCampo(service,dictionary,"email","email")
    rellenarCampo(service,dictionary,"web","web")
    rellenarCampo(service,dictionary,"phone","phone")
    dictionary["horario"]=None
    extra_data=service.getElementsByTagName("extradata")[0]

    for item in extra_data.getElementsByTagName("item"):
        if(item.hasAttribute("name")and item.getAttribute("name")=="Horario"):
            dictionary["horario"]=html.unescape(item.childNodes[0].data)
            break
    return dictionary
def info_restaurante(filename, name):
    ArbolDOM=xml.dom.minidom.parse(filename)
    services=ArbolDOM.documentElement.getElementsByTagName("service")
    for service in services:
        aux=service.getElementsByTagName("name")[0]
        current_name=html.unescape(aux.childNodes[0].data).strip()
        if(current_name!=name):
            continue
        return rellenarDiccionario(service,current_name)
    return None
# for clave, valor in info_restaurante("restaurantes_v1_es.xml","Hasaku Nikkei").items():
#     print(f'{clave}: {valor}')

In [ ]:
from geopy.geocoders import Nominatim
from geopy import distance
def busqueda_cercania(filename, lugar, n):
    ArbolDOM=xml.dom.minidom.parse(filename)
    services=ArbolDOM.documentElement.getElementsByTagName("service")    
    geolocator = Nominatim(user_agent="GIW_pr2")
    origin=geolocator.geocode(lugar,addressdetails=True)
    lista=[]
    for service in services:
        lugar=(service.getElementsByTagName("latitude")[0].childNodes[0].data,
                service.getElementsByTagName("longitude")[0].childNodes[0].data)
        distancia_al_restaurante=distance.distance((origin.latitude,origin.longitude), lugar).km
        if(distancia_al_restaurante<n):
            lista.append((distancia_al_restaurante,html.unescape(service.getElementsByTagName("name")[0].childNodes[0].data)))
    lista.sort(key=lambda restaurante:restaurante[0])
    return lista
for r in busqueda_cercania('restaurantes_v1_es.xml','Profesor José García Santesmases 9, Madrid, España', 3):
    print(r)